## แชทบอท Phi 3 Mini 4K แบบโต้ตอบพร้อม Whisper

### บทนำ:
แชทบอท Phi 3 Mini 4K แบบโต้ตอบเป็นเครื่องมือที่ช่วยให้ผู้ใช้สามารถโต้ตอบกับ Microsoft Phi 3 Mini 4K instruct demo ผ่านการป้อนข้อมูลด้วยข้อความหรือเสียง แชทบอทนี้สามารถใช้งานได้หลากหลาย เช่น การแปลภาษา การอัปเดตสภาพอากาศ และการรวบรวมข้อมูลทั่วไป


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


[สร้าง Huggingface Access Token ของคุณ](https://huggingface.co/settings/tokens)

สร้างโทเค็นใหม่  
ตั้งชื่อใหม่  
เลือกสิทธิ์การเขียน  
คัดลอกโทเค็นและเก็บไว้ในที่ปลอดภัย


Python และโค้ดนี้มีหน้าที่หลักสองอย่าง: การนำเข้าโมดูล `os` และการตั้งค่าตัวแปรสภาพแวดล้อม

1. การนำเข้าโมดูล `os`:
   - โมดูล `os` ใน Python ช่วยให้สามารถโต้ตอบกับระบบปฏิบัติการได้ โดยให้คุณทำงานที่เกี่ยวข้องกับระบบปฏิบัติการ เช่น การเข้าถึงตัวแปรสภาพแวดล้อม การทำงานกับไฟล์และไดเรกทอรี เป็นต้น
   - ในโค้ดนี้ โมดูล `os` ถูกนำเข้าด้วยคำสั่ง `import` ซึ่งทำให้ฟังก์ชันการทำงานของโมดูล `os` พร้อมใช้งานในสคริปต์ Python ปัจจุบัน

2. การตั้งค่าตัวแปรสภาพแวดล้อม:
   - ตัวแปรสภาพแวดล้อมคือตัวแปรที่สามารถเข้าถึงได้โดยโปรแกรมที่ทำงานบนระบบปฏิบัติการ เป็นวิธีการเก็บค่าการตั้งค่าหรือข้อมูลอื่น ๆ ที่สามารถใช้ร่วมกันโดยหลายโปรแกรม
   - ในโค้ดนี้ มีการตั้งค่าตัวแปรสภาพแวดล้อมใหม่โดยใช้พจนานุกรม `os.environ` โดยคีย์ของพจนานุกรมคือ `'HF_TOKEN'` และค่าถูกกำหนดจากตัวแปร `HUGGINGFACE_TOKEN`
   - ตัวแปร `HUGGINGFACE_TOKEN` ถูกกำหนดไว้ด้านบนของโค้ดนี้ และได้รับค่าข้อความ `"hf_**************"` โดยใช้ไวยากรณ์ `#@param` ซึ่งไวยากรณ์นี้มักใช้ใน Jupyter notebooks เพื่อให้ผู้ใช้สามารถป้อนข้อมูลและตั้งค่าพารามิเตอร์ได้โดยตรงในอินเทอร์เฟซของโน้ตบุ๊ก
   - เมื่อมีการตั้งค่าตัวแปรสภาพแวดล้อม `'HF_TOKEN'` ตัวแปรนี้สามารถถูกเข้าถึงได้โดยส่วนอื่น ๆ ของโปรแกรมหรือโปรแกรมอื่น ๆ ที่ทำงานบนระบบปฏิบัติการเดียวกัน

โดยรวมแล้ว โค้ดนี้นำเข้าโมดูล `os` และตั้งค่าตัวแปรสภาพแวดล้อมชื่อ `'HF_TOKEN'` ด้วยค่าที่กำหนดในตัวแปร `HUGGINGFACE_TOKEN`


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

โค้ดนี้เป็นตัวอย่างของฟังก์ชันที่ชื่อว่า `clear_output` ซึ่งใช้สำหรับล้างผลลัพธ์ของเซลล์ปัจจุบันใน Jupyter Notebook หรือ IPython มาดูรายละเอียดของโค้ดและทำความเข้าใจการทำงานของมัน:

ฟังก์ชัน `clear_output` รับพารามิเตอร์หนึ่งตัวที่ชื่อว่า `wait` ซึ่งเป็นค่าบูลีน โดยค่าเริ่มต้นของ `wait` ถูกตั้งไว้เป็น `False` พารามิเตอร์นี้ใช้กำหนดว่าฟังก์ชันควรรอจนกว่าผลลัพธ์ใหม่จะพร้อมสำหรับแทนที่ผลลัพธ์เดิมก่อนที่จะล้างผลลัพธ์หรือไม่

ตัวฟังก์ชันเองถูกใช้สำหรับล้างผลลัพธ์ของเซลล์ปัจจุบัน ใน Jupyter Notebook หรือ IPython เมื่อเซลล์สร้างผลลัพธ์ เช่น ข้อความที่พิมพ์ออกมาหรือกราฟ ผลลัพธ์นั้นจะถูกแสดงอยู่ด้านล่างเซลล์ ฟังก์ชัน `clear_output` ช่วยให้คุณสามารถล้างผลลัพธ์เหล่านั้นได้

การทำงานของฟังก์ชันไม่ได้ถูกระบุในโค้ดตัวอย่างนี้ โดยมีเครื่องหมายจุดไข่ปลา (...) แสดงถึงตำแหน่งที่ควรจะมีโค้ดจริงสำหรับการล้างผลลัพธ์ การทำงานของฟังก์ชันอาจเกี่ยวข้องกับการใช้งาน API ของ Jupyter Notebook หรือ IPython เพื่อทำการลบผลลัพธ์ที่มีอยู่ในเซลล์

โดยรวมแล้ว ฟังก์ชันนี้เป็นเครื่องมือที่สะดวกสำหรับการล้างผลลัพธ์ของเซลล์ปัจจุบันใน Jupyter Notebook หรือ IPython ซึ่งช่วยให้การจัดการและอัปเดตผลลัพธ์ที่แสดงในระหว่างการเขียนโค้ดแบบโต้ตอบเป็นไปได้ง่ายขึ้น


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

ดำเนินการแปลงข้อความเป็นเสียง (TTS) โดยใช้บริการ Edge TTS มาดูการทำงานของฟังก์ชันที่เกี่ยวข้องทีละขั้นตอน:

1. `calculate_rate_string(input_value)`: ฟังก์ชันนี้รับค่าที่ป้อนเข้ามาและคำนวณอัตราความเร็วของเสียง TTS ค่า input_value แสดงถึงความเร็วที่ต้องการของเสียง โดยค่าที่เป็น 1 หมายถึงความเร็วปกติ ฟังก์ชันจะคำนวณอัตราความเร็วโดยการลบ 1 จากค่า input_value คูณด้วย 100 และกำหนดเครื่องหมาย (+ หรือ -) ตามว่าค่า input_value มากกว่าหรือเท่ากับ 1 หรือไม่ ฟังก์ชันจะคืนค่าอัตราความเร็วในรูปแบบ "{sign}{rate}"

2. `make_chunks(input_text, language)`: ฟังก์ชันนี้รับข้อความและภาษาที่ป้อนเข้ามาเป็นพารามิเตอร์ มันจะแบ่งข้อความออกเป็นส่วนย่อยตามกฎเฉพาะของภาษา ในการใช้งานนี้ หากภาษาคือ "English" ฟังก์ชันจะแบ่งข้อความที่จุด (".") และลบช่องว่างที่ขึ้นต้นหรือท้ายออก จากนั้นจะเพิ่มจุดต่อท้ายแต่ละส่วนและคืนค่ารายการของส่วนที่กรองแล้ว

3. `tts_file_name(text)`: ฟังก์ชันนี้สร้างชื่อไฟล์สำหรับไฟล์เสียง TTS ตามข้อความที่ป้อนเข้ามา มันจะทำการเปลี่ยนแปลงข้อความหลายอย่าง เช่น ลบจุดที่ท้ายข้อความ (ถ้ามี) เปลี่ยนข้อความเป็นตัวพิมพ์เล็ก ลบช่องว่างที่ขึ้นต้นและท้าย และแทนที่ช่องว่างด้วยเครื่องหมายขีดล่าง จากนั้นจะตัดข้อความให้มีความยาวสูงสุด 25 ตัวอักษร (ถ้ายาวกว่า) หรือใช้ข้อความเต็มถ้าไม่มีข้อความ สุดท้ายจะสร้างสตริงสุ่มโดยใช้โมดูล [`uuid`] และรวมเข้ากับข้อความที่ตัดแล้วเพื่อสร้างชื่อไฟล์ในรูปแบบ "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"

4. `merge_audio_files(audio_paths, output_path)`: ฟังก์ชันนี้รวมไฟล์เสียงหลายไฟล์เป็นไฟล์เสียงเดียว มันรับรายการของเส้นทางไฟล์เสียงและเส้นทางไฟล์ผลลัพธ์เป็นพารามิเตอร์ ฟังก์ชันจะเริ่มต้นวัตถุ `AudioSegment` ว่างเปล่าที่เรียกว่า [`merged_audio`] จากนั้นจะวนลูปผ่านแต่ละเส้นทางไฟล์เสียง โหลดไฟล์เสียงโดยใช้เมธอด `AudioSegment.from_file()` จากไลบรารี `pydub` และเพิ่มไฟล์เสียงปัจจุบันเข้าไปในวัตถุ [`merged_audio`] สุดท้ายจะส่งออกไฟล์เสียงที่รวมแล้วไปยังเส้นทางไฟล์ที่ระบุในรูปแบบ MP3

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: ฟังก์ชันนี้ดำเนินการ TTS โดยใช้บริการ Edge TTS มันรับรายการของข้อความที่แบ่งส่วน ความเร็วของเสียง ชื่อเสียง และเส้นทางการบันทึกเป็นพารามิเตอร์ หากจำนวนส่วนมากกว่า 1 ฟังก์ชันจะสร้างไดเรกทอรีสำหรับเก็บไฟล์เสียงของแต่ละส่วน จากนั้นจะวนลูปผ่านแต่ละส่วน สร้างคำสั่ง Edge TTS โดยใช้ฟังก์ชัน `calculate_rate_string()` ชื่อเสียง และข้อความของแต่ละส่วน และดำเนินการคำสั่งโดยใช้ฟังก์ชัน `os.system()` หากการดำเนินการคำสั่งสำเร็จ จะเพิ่มเส้นทางของไฟล์เสียงที่สร้างขึ้นในรายการ หลังจากประมวลผลทุกส่วนแล้ว จะรวมไฟล์เสียงแต่ละไฟล์โดยใช้ฟังก์ชัน `merge_audio_files()` และบันทึกไฟล์เสียงที่รวมแล้วไปยังเส้นทางการบันทึกที่ระบุ หากมีเพียงส่วนเดียว จะสร้างคำสั่ง Edge TTS โดยตรงและบันทึกเสียงไปยังเส้นทางการบันทึก สุดท้ายจะคืนค่าเส้นทางการบันทึกของไฟล์เสียงที่สร้างขึ้น

6. `random_audio_name_generate()`: ฟังก์ชันนี้สร้างชื่อไฟล์เสียงแบบสุ่มโดยใช้โมดูล [`uuid`] มันสร้าง UUID แบบสุ่ม แปลงเป็นสตริง เลือกตัวอักษร 8 ตัวแรก เพิ่มนามสกุล ".mp3" และคืนค่าชื่อไฟล์เสียงแบบสุ่ม

7. `talk(input_text)`: ฟังก์ชันนี้เป็นจุดเริ่มต้นหลักสำหรับการดำเนินการ TTS มันรับข้อความที่ป้อนเข้ามาเป็นพารามิเตอร์ ก่อนอื่นจะตรวจสอบความยาวของข้อความที่ป้อนเข้ามาเพื่อกำหนดว่าข้อความนั้นเป็นประโยคยาวหรือไม่ (มากกว่าหรือเท่ากับ 600 ตัวอักษร) ตามความยาวและค่าของตัวแปร `translate_text_flag` มันจะกำหนดภาษาและสร้างรายการของข้อความที่แบ่งส่วนโดยใช้ฟังก์ชัน `make_chunks()` จากนั้นจะสร้างเส้นทางการบันทึกสำหรับไฟล์เสียงโดยใช้ฟังก์ชัน `random_audio_name_generate()` สุดท้ายจะเรียกใช้ฟังก์ชัน `edge_free_tts()` เพื่อดำเนินการ TTS และคืนค่าเส้นทางการบันทึกของไฟล์เสียงที่สร้างขึ้น

โดยรวมแล้ว ฟังก์ชันเหล่านี้ทำงานร่วมกันเพื่อแบ่งข้อความที่ป้อนเข้ามาเป็นส่วนย่อย สร้างชื่อไฟล์สำหรับไฟล์เสียง ดำเนินการ TTS โดยใช้บริการ Edge TTS และรวมไฟล์เสียงแต่ละไฟล์เป็นไฟล์เสียงเดียว


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

การใช้งานของสองฟังก์ชัน: convert_to_text และ run_text_prompt รวมถึงการประกาศของสองคลาส: str และ Audio

ฟังก์ชัน convert_to_text รับค่า audio_path เป็นอินพุตและแปลงเสียงเป็นข้อความโดยใช้โมเดลที่เรียกว่า whisper_model ฟังก์ชันนี้จะตรวจสอบก่อนว่าตัวแปร gpu ถูกตั้งค่าเป็น True หรือไม่ หากเป็น True จะใช้ whisper_model พร้อมกับพารามิเตอร์บางอย่าง เช่น word_timestamps=True, fp16=True, language='English', และ task='translate' หาก gpu เป็น False จะใช้ whisper_model พร้อมกับ fp16=False ข้อความที่ได้จากการถอดเสียงจะถูกบันทึกลงในไฟล์ชื่อ 'scan.txt' และส่งคืนเป็นข้อความ

ฟังก์ชัน run_text_prompt รับค่า message และ chat_history เป็นอินพุต โดยใช้ฟังก์ชัน phi_demo เพื่อสร้างการตอบกลับจากแชทบอทตามข้อความที่ป้อนเข้าไป การตอบกลับที่สร้างขึ้นจะถูกส่งต่อไปยังฟังก์ชัน talk ซึ่งแปลงการตอบกลับเป็นไฟล์เสียงและส่งคืนเส้นทางของไฟล์ คลาส Audio ถูกใช้เพื่อแสดงและเล่นไฟล์เสียง เสียงจะถูกแสดงโดยใช้ฟังก์ชัน display จากโมดูล IPython.display และออบเจ็กต์ Audio ถูกสร้างขึ้นพร้อมกับพารามิเตอร์ autoplay=True เพื่อให้เสียงเริ่มเล่นโดยอัตโนมัติ chat_history จะถูกอัปเดตด้วยข้อความที่ป้อนและการตอบกลับที่สร้างขึ้น และส่งคืนสตริงว่างพร้อมกับ chat_history ที่อัปเดตแล้ว

คลาส str เป็นคลาสที่มีอยู่ใน Python ซึ่งแสดงถึงลำดับของตัวอักษร โดยมีเมธอดต่าง ๆ สำหรับการจัดการและการทำงานกับสตริง เช่น capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill และอื่น ๆ เมธอดเหล่านี้ช่วยให้สามารถดำเนินการต่าง ๆ เช่น การค้นหา การแทนที่ การจัดรูปแบบ และการจัดการสตริงได้

คลาส Audio เป็นคลาสที่กำหนดเองซึ่งแสดงถึงออบเจ็กต์เสียง โดยใช้เพื่อสร้างเครื่องเล่นเสียงในสภาพแวดล้อม Jupyter Notebook คลาสนี้รับพารามิเตอร์ต่าง ๆ เช่น data, filename, url, embed, rate, autoplay และ normalize พารามิเตอร์ data สามารถเป็น numpy array, ลิสต์ของตัวอย่าง, สตริงที่แสดงถึงชื่อไฟล์หรือ URL หรือข้อมูล PCM ดิบ พารามิเตอร์ filename ใช้เพื่อระบุไฟล์ในเครื่องเพื่อโหลดข้อมูลเสียง และพารามิเตอร์ url ใช้เพื่อระบุ URL เพื่อดาวน์โหลดข้อมูลเสียง พารามิเตอร์ embed กำหนดว่าข้อมูลเสียงควรถูกฝังโดยใช้ data URI หรืออ้างอิงจากแหล่งที่มาเดิม พารามิเตอร์ rate ระบุอัตราการสุ่มตัวอย่างของข้อมูลเสียง พารามิเตอร์ autoplay กำหนดว่าเสียงควรเริ่มเล่นโดยอัตโนมัติหรือไม่ พารามิเตอร์ normalize ระบุว่าข้อมูลเสียงควรถูกปรับให้เป็นช่วงสูงสุดที่เป็นไปได้หรือไม่ คลาส Audio ยังมีเมธอด เช่น reload เพื่อโหลดข้อมูลเสียงใหม่จากไฟล์หรือ URL และแอตทริบิวต์ เช่น src_attr, autoplay_attr และ element_id_attr เพื่อดึงแอตทริบิวต์ที่เกี่ยวข้องสำหรับออบเจ็กต์เสียงใน HTML

โดยรวมแล้ว ฟังก์ชันและคลาสเหล่านี้ถูกใช้เพื่อแปลงเสียงเป็นข้อความ สร้างการตอบกลับเสียงจากแชทบอท และแสดงและเล่นเสียงในสภาพแวดล้อม Jupyter Notebook


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้องมากที่สุด แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษามืออาชีพ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดที่เกิดจากการใช้การแปลนี้
